In [ ]:
from pathlib import Path
import sys

ROOT = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / "scripts").is_dir() or (p / "data").is_dir()), None)
if ROOT is None:
    raise RuntimeError("Repo-Root not found (expected folder 'scripts' or 'data').")
sys.path.insert(0, str(ROOT))

DATA_DIR = ROOT / "data"
DRF_DIRS_BIG = [(DATA_DIR / "drf_big" / f"precomputed_drf_{m}", m) for m in ("edge", "vertex", "sp")]
DRF_DIRS_SMALL = [(DATA_DIR / "drf_small" / f"precomputed_drf_{m}", m) for m in ("edge", "vertex", "sp")]
ITS_DIRS_BIG = [(DATA_DIR / "its_big" / f"precomputed_its_{m}", m) for m in ("edge", "vertex", "sp")]
ITS_DIRS_SMALL = [(DATA_DIR / "its_small" / f"precomputed_its_{m}", m) for m in ("edge", "vertex", "sp")]

# WP3 — Kernel-based Classification (SVM)

This notebook implements kernel inner products on precomputed hashed feature sets and runs
SVM classification for DRF–WL and ITS–WL across different feature types (vertex/edge/shortest-path),
dataset sizes, numbers of classes, and train/test splits.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
import pickle
from pathlib import Path
from collections import Counter

#local imports
from scripts.wp3.wp3_loader import (
    load_precomputed_features,
    build_subset_index,
    load_precomputed_features_select,
    choose_subsets_with_fixed_classes,
    choose_subsets_with_at_least_k_common_classes,
    common_classes_across_subsets,
)

from scripts.wp3.wp3_kernel import (
    build_kernel_matrix_from_loaded, 
    kernel_matrix_stats,
    kernel_multiset_intersection,
    build_kernel_matrix_from_loaded, 
    kernel_matrix_stats,
)

from scripts.wp3.wp3_svm import (
    train_svm_from_precomputed_dir,
)

from scripts.wp3.wp3_plots import (
    plot_experiment_results,   
)

## 1) Paths to precomputed feature directories

We load precomputed feature representations (stored as `.pkl`) for:
- DRF–WL: reactant/product difference features
- ITS–WL: features from the ITS reaction graph

Each representation is available for three feature modes: vertex, edge, shortest-path.

### Load DRF–WL Features
Load precomputed DRF–WL feature sets and reaction class labels for kernel-based classification.

In [ ]:

X_drf, y_drf = {}, {}
for path, mode in DRF_DIRS_BIG:  # ACHTUNG: Reihenfolge (path, mode)
    assert path.exists(), f"Pfad nicht gefunden: {path}"
    X, y = load_precomputed_features(path, feature_key="drf_wl")
    X_drf[mode] = X
    y_drf[mode] = y
    print(f"\nLoaded DRF features ({mode}) from {path}")
    print("Number of reactions:", len(X))
    print("Number of classes:", len(set(y)))

### Load ITS–WL Features
Load precomputed ITS–WL feature sets and reaction class labels derived from the ITS graph.

In [ ]:
X_its = {}
y_its = {}
for path, mode in ITS_DIRS_BIG:  # ACHTUNG: Reihenfolge (path, mode)
    assert path.exists(), f"Pfad nicht gefunden: {path}"
    X, y = load_precomputed_features(path, feature_key="its_wl")
    X_its[mode] = X
    y_its[mode] = y
    print(f"\nLoaded ITS features ({mode}) from {path}")
    print("Number of reactions:", len(X))
    print("Number of classes:", len(set(y)))


The output confirms that all precomputed DRF–WL feature representations
(edge, vertex, and shortest-path) were loaded successfully. Each representation
contains the full dataset of 50,000 reactions across 50 reaction classes,
providing a consistent basis for kernel computation and classification.

## 2) Kernel inner product on hash sets

The lab definition reduces all kernels to counting common elements of two hashed feature sets.
Given two reactions with feature hash sets \(S_G, S_H\), the kernel is:
\[
k(G,H) = |S_G \cap S_H|
\]

Our precomputed features are stored as Counters. For the required hashset kernel, we use the Counter keys.

Ein Kernel ist eine Funktion, die sagt, wie ähnlich zwei Reaktionen sind.

### Kernel sanity check (DRF–WL)

We verify that the multiset kernel produces meaningful similarities on the precomputed DRF–WL feature multisets.  
Self-similarity \(k(x,x)\) is clearly positive, and different reactions can still share a non-zero overlap, indicating common reaction-change patterns captured by DRF–WL.

In [ ]:
mode = "edge"   # "edge" | "vertex" | "sp"
X = X_its[mode]  # oder X_drf[mode]

# finde erstes Paar mit k>0
for i in range(len(X)):
    if len(X[i]) == 0:
        continue
    for j in range(i + 1, len(X)):
        if len(X[j]) == 0:
            continue
        k = kernel_multiset_intersection(X[i], X[j])
        if k > 0:
            print("Found non-zero kernel at:", i, j, "value:", k)
            break
    else:
        continue
    break


In [ ]:
# Finde ein nicht-leeres Paar
for i in range(len(X)):
    if len(X[i]) == 0:
        continue
    for j in range(i+1, len(X)):
        if len(X[j]) == 0:
            continue
        k = kernel_multiset_intersection(X[i], X[j])
        if k > 0:
            print("Found non-zero kernel at:", i, j, "value:", k)
            break
    else:
        continue
    break

### Kernel Matrix Construction

To apply kernel-based classification, the pairwise similarities between all reactions are computed and stored in a kernel matrix. Each entry \(K_{ij}\) represents the multiset kernel value between reactions \(i\) and \(j\). This matrix serves as the direct input for training a Support Vector Machine with a precomputed kernel.

### DRF–WL Kernel Matrix (edge features)

This heatmap visualizes the kernel matrix computed using the DRF–WL edge kernel for a subset of reactions.
Each entry \(K_{ij}\) represents the multiset intersection between the DRF–WL feature representations of reaction \(i\) and reaction \(j\).

The bright diagonal indicates high self-similarity, as each reaction shares all its features with itself.
Most off-diagonal entries are close to zero, which reflects the sparsity of the DRF representation:  
DRF removes all static molecular structure and retains only features corresponding to reaction-specific changes.

Non-zero off-diagonal values highlight pairs of reactions that share similar bond-change patterns.
This confirms that the DRF–WL kernel captures meaningful similarities between reactions while remaining highly selective.

In [ ]:
mode = "edge"   # "edge" | "vertex" | "sp"
n = 200

K_drf, y_small = build_kernel_matrix_from_loaded(
    X_drf, y_drf,
    mode=mode,
    n=n,
)

stats = kernel_matrix_stats(K_drf)
print("Kernel matrix stats:", stats)

fig = px.imshow(
    K_drf,
    title=f"Kernel Matrix Heatmap (DRF–WL {mode}, n={n})",
    aspect="auto",
)
fig.show()

**Figure (DRF–WL):** Kernel matrix heatmap computed using the DRF–WL edge kernel.
Each entry \(K_{ij}\) represents the multiset intersection between the DRF–WL feature representations of reactions \(i\) and \(j\).
The diagonal indicates self-similarity, while off-diagonal values are mostly close to zero.
This sparsity reflects the DRF representation, which removes static molecular structure and retains only reaction-specific changes.
Non-zero off-diagonal entries therefore highlight reactions with similar bond-change patterns.

#### Error Handling

In [ ]:
path = DATA_DIR / "drf_small/precomputed_drf_edge"
pkl = next(path.glob("*.pkl"))   # ← HIER ist next() richtig
obj = pickle.load(open(pkl, "rb"))

print("Keys:", obj.keys())
print("n_errors:", obj["meta"]["n_errors"])
print("First error:", obj["errors"][:1])
print("First feature:", type(obj["drf_wl"][0]), obj["drf_wl"][0])

In [ ]:

DIR = DATA_DIR / "drf_small/precomputed_drf_edge"  # <- GENAU der Ordner, den du lädst
pkl = sorted(DIR.glob("*.pkl"))[0]
print("Inspecting:", pkl)

with open(pkl, "rb") as f:
    obj = pickle.load(f)

print("Keys:", obj.keys())
print("Meta n_rows:", obj["meta"]["n_rows"])
print("Meta n_errors:", obj["meta"]["n_errors"])
print("First error (if any):", obj["errors"][:1])

# Jetzt das wichtigste:
X = obj["drf_wl"]
empty = sum(1 for c in X if len(c) == 0)
print("Empty counters:", empty, "/", len(X))

# Beispiel suchen
for i, c in enumerate(X):
    if len(c) > 0:
        print("First non-empty at idx:", i, "items:", len(c), "total:", sum(c.values()))
        print("Sample:", list(c.items())[:5])
        break
else:
    print("ALL COUNTERS ARE EMPTY in this PKL.")


In [ ]:
pkls = list(path.glob("*.pkl"))
print("Found PKLs:", len(pkls))

pkl = pkls[0]
obj = pickle.load(open(pkl, "rb"))

print("n_errors:", obj["meta"]["n_errors"])
print("empty:", sum(1 for c in obj["drf_wl"] if len(c)==0), "/", len(obj["drf_wl"]))
print("example total count:", sum(obj["drf_wl"][0].values()))

### ITS–WL Kernel Matrix (edge features)

This heatmap shows the kernel matrix computed using the ITS–WL edge kernel.
Here, reactions are represented by Weisfeiler–Lehman features extracted from the Imaginary Transition State (ITS) graph.

Compared to DRF–WL, the ITS–WL kernel produces a denser similarity structure.
This is expected, as the ITS graph encodes the full combined structure of reactants and products, including unchanged molecular context.

The diagonal again represents self-similarity, while the richer off-diagonal structure indicates that many reactions share common substructures.
As a result, ITS–WL captures broader structural similarity between reactions, not only the explicit reaction center.

In [ ]:
mode = "edge"
n = 200

K_its, y_its_small = build_kernel_matrix_from_loaded(
    X_its, y_its,
    mode=mode,
    n=n,
)

print("ITS kernel matrix stats:", kernel_matrix_stats(K_its))

fig = px.imshow(
    K_its,
    title=f"Kernel Matrix Heatmap (ITS–WL {mode}, n={n})",
    aspect="auto",
)
fig.show()

**Figure (ITS–WL):** Kernel matrix heatmap computed using the ITS–WL edge kernel.
Each entry \(K_{ij}\) corresponds to the multiset intersection of Weisfeiler–Lehman features extracted from the Imaginary Transition State graphs.
Compared to DRF–WL, the ITS–WL kernel exhibits a denser similarity structure, as the ITS graph encodes the full molecular context of reactants and products.
Off-diagonal similarities reflect shared structural motifs beyond the reaction center.

### Comparison of DRF–WL and ITS–WL Kernel Matrices

The DRF–WL and ITS–WL kernel matrices reveal complementary notions of reaction similarity.
DRF–WL focuses exclusively on reaction-specific changes by computing the symmetric difference between reactant and product features.
As a result, the corresponding kernel matrix is sparse, with non-zero similarities only for reactions that share similar bond-change patterns.

In contrast, ITS–WL operates on the Imaginary Transition State graph, which encodes the full structural context of both reactants and products.
This leads to a denser kernel matrix, as reactions may share common substructures even if their reaction centers differ.

Consequently, DRF–WL provides a highly selective notion of similarity tailored to reaction mechanisms,
whereas ITS–WL captures broader structural resemblance between reactions.
Both representations are therefore suitable for different aspects of reaction classification.

**Figure:** Kernel matrix heatmaps for DRF–WL (bottom) and ITS–WL (top) using edge-based Weisfeiler–Lehman features.
Each entry \(K_{ij}\) corresponds to the multiset intersection between the feature representations of reactions \(i\) and \(j\).
The diagonal indicates self-similarity, while off-diagonal values reflect shared structural or reaction-specific features.
DRF–WL produces a sparse kernel emphasizing reaction changes, whereas ITS–WL yields a denser kernel capturing overall structural similarity.

In [ ]:

def upper_triangle_values(K):
    n = K.shape[0]
    return K[np.triu_indices(n, k=1)]

vals_drf = upper_triangle_values(K_drf)  # DRF Kernel-Matrix
vals_its = upper_triangle_values(K_its)  # ITS Kernel-Matrix

fig = px.histogram(
    x=[vals_drf, vals_its],
    labels={"value": "Kernel value", "variable": "Kernel"},
    nbins=50,
    opacity=0.6,
    title="Distribution of Kernel Values: DRF–WL vs ITS–WL",
)

fig.data[0].name = "DRF–WL"
fig.data[1].name = "ITS–WL"
fig.show()

**Figure:** Distribution of off-diagonal kernel values for DRF–WL and ITS–WL.
DRF–WL produces a highly sparse similarity distribution with many zero entries, reflecting its focus on reaction-specific changes.
In contrast, ITS–WL yields a broader distribution, capturing shared structural context between reactions.

## SVM Classification with a Custom Reaction Kernel

An SVM classifier was trained using a custom kernel based on the multiset intersection of reaction features.
Since the kernel operates on pairs of reactions rather than explicit feature vectors, the kernel matrix was precomputed and passed to the SVM using `kernel="precomputed"`.
All classification experiments are conducted using precomputed kernel feature representations.
This enables a fair comparison between DRF–WL and ITS–WL kernels, as the same SVM configuration
and training procedure is applied to both representations.


To systematically evaluate kernel variants, we run the same SVM setup for each feature mode separately.
This yields comparable accuracies for edge-, vertex-, and shortest-path-based WL representations without mixing feature spaces.

### 0) Subsets vorbereiten nach passenden Klassen


#### Results speichern

In [ ]:
from pathlib import Path
import sys

# Repo-Root robust finden (da wo "data" oder "scripts" liegt)
ROOT = next(
    (p for p in [Path.cwd(), *Path.cwd().parents]
     if (p / "data").is_dir() or (p / "scripts").is_dir()),
    None
)
if ROOT is None:
    raise RuntimeError("Repo-Root not found (expected folder 'data' or 'scripts').")

DATA_DIR = ROOT / "data"
print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)

# Deine Precompute-Ordner (SMALL-Beispiel)
DRF_DIRS = {
    "edge":   DATA_DIR / "drf_small" / "precomputed_drf_edge",
    "vertex": DATA_DIR / "drf_small" / "precomputed_drf_vertex",
    "sp":     DATA_DIR / "drf_small" / "precomputed_drf_sp",
}
ITS_DIRS = {
    "edge":   DATA_DIR / "its_small" / "precomputed_its_edge",
    "vertex": DATA_DIR / "its_small" / "precomputed_its_vertex",
    "sp":     DATA_DIR / "its_small" / "precomputed_its_sp",
}

for k,v in DRF_DIRS.items():
    print("DRF", k, "->", v)
for k,v in ITS_DIRS.items():
    print("ITS", k, "->", v)

In [ ]:
def count_pkls(p: Path) -> int:
    return len(list(p.glob("*.pkl")))

print("\n--- PKL counts ---")
for mode, p in DRF_DIRS.items():
    print("DRF", mode, ":", count_pkls(p))
for mode, p in ITS_DIRS.items():
    print("ITS", mode, ":", count_pkls(p))

In [ ]:
# ==========================================
# Results collector
# ==========================================

results = []

def add_result(
    *,
    tag,
    kernel,
    mode,
    n,
    test_size,
    C,
    seed,
    res,
    subset_ids=None,
    precomp_dir=None,
    feature_key=None,
):
    """
    Adds one experiment result to global list `results`.

    Works with:
      - dict results: res["accuracy"]
      - object results: res.accuracy
    """

    # --- robustly extract accuracy ---
    if isinstance(res, dict):
        acc = res.get("accuracy", None)
    else:
        acc = getattr(res, "accuracy", None)

    if acc is None:
        raise ValueError(
            "Could not extract accuracy from 'res'. "
            "Expected dict with key 'accuracy' or object with attribute '.accuracy'."
        )

    results.append({
        "tag": tag,
        "kernel": kernel,
        "mode": mode,
        "precomp_dir": str(precomp_dir) if precomp_dir is not None else None,
        "feature_key": feature_key,
        "subset_ids": list(subset_ids) if subset_ids is not None else None,
        "n": int(n),
        "test_size": float(test_size),
        "C": float(C),
        "seed": int(seed),
        "accuracy": float(acc),
    })

In [ ]:
# ==========================================
# Helper: find available subset IDs in a dir
# ==========================================

def available_subset_ids(precomp_dir: str | Path) -> list[int]:
    precomp_dir = Path(precomp_dir)
    ids = []
    for fp in precomp_dir.glob("subset_*.pkl"):
        # subset_001....pkl -> 1
        sid = int(fp.name.split("subset_")[1][:3])
        ids.append(sid)
    return sorted(set(ids))


# ==========================================
# Option 1 ONLY: choose many subsets robustly
# ==========================================

def make_option1_config(
    *,
    drf_edge_dir: str | Path,
    its_edge_dir: str | Path,
    ref_take: int = 20,
    k: int = 1,
    take_subsets: int = 10,
    min_per_class: int = 20,
) -> dict:
    drf_edge_dir = Path(drf_edge_dir)
    its_edge_dir = Path(its_edge_dir)

    # only keep subset ids that exist in both
    common_ids = sorted(set(available_subset_ids(drf_edge_dir)) & set(available_subset_ids(its_edge_dir)))
    if not common_ids:
        raise FileNotFoundError("No common subset PKLs between DRF and ITS edge dirs.")

    drf_index = build_subset_index(drf_edge_dir)  # subset_id -> {class: count}
    its_index = build_subset_index(its_edge_dir)

    # build reference class pool from many subsets (more stable)
    class_pool = Counter()
    for sid in common_ids[:min(30, len(common_ids))]:
        class_pool.update(drf_index[sid].keys())
    ref_classes = [c for c, _ in class_pool.most_common(ref_take)]

    # subsets with >=k overlap with ref_classes
    drf_ok = choose_subsets_with_at_least_k_common_classes(drf_index, ref_classes, k=k, min_per_class=min_per_class)
    its_ok = choose_subsets_with_at_least_k_common_classes(its_index, ref_classes, k=k, min_per_class=min_per_class)

    subset_ids = sorted(set(drf_ok) & set(its_ok))[:take_subsets]
    if not subset_ids:
        # fallback: just take the first common ids
        subset_ids = common_ids[:min(take_subsets, len(common_ids))]

    return {
        "name": f"opt1_overlap_k{k}",
        "subset_ids": subset_ids,
        "ref_classes": ref_classes,
        "ref_take": ref_take,
        "k": k,
        "take_subsets": take_subsets,
        "min_per_class": min_per_class,
    }

### 1) Baseline Comparison: DRF–WL vs ITS–WL

In this experiment, DRF–WL and ITS–WL kernels are compared under identical conditions to provide a fair baseline.
All parameters are fixed (feature mode, dataset size, train/test split, and SVM regularization), and only the
graph representation differs. This allows us to directly assess the impact of reaction-based versus structure-based
graph representations on classification performance.

In [ ]:
#opt1 = make_option1_config(..., ref_take=30, k=1, take_subsets=20)

In [ ]:
def load_dir_for_subsets(precomp_dir, feature_key, subset_ids):
    # falls dein loader subset_ids direkt kann, nutze den.
    # ansonsten: lade alles und filtere später.
    X, y = load_precomputed_features(precomp_dir, feature_key=feature_key)
    return X, y

In [ ]:
results = []

# Settings
C = 1.0
seed = 42
test_size = 0.2
n = 600

# OPTION 1: einfach gemeinsame subset_ids nehmen (ohne Klassen-Zwang)
common_subset_ids = sorted(set(available_subset_ids(DRF_DIRS["edge"])) & set(available_subset_ids(ITS_DIRS["edge"])))
if not common_subset_ids:
    raise FileNotFoundError("Keine gemeinsamen subset_XXX PKLs zwischen DRF(edge) und ITS(edge). Prüfe die Ordnerpfade.")

subset_ids = common_subset_ids[:10]   # <- NIMM MEHR, wenn du mehr willst (z.B. 30, 50, ...)
print("Chosen subset_ids:", subset_ids)

# DRF edge baseline
res = train_svm_from_precomputed_dir(
    precomp_dir=DRF_DIRS["edge"],
    feature_key="drf_wl",
    subset_ids=subset_ids,
    n=n, test_size=test_size, C=C, seed=seed,
    verbose=True,
)
add_result(
    tag="S4_baseline",
    kernel="DRF–WL",
    mode="edge",
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
)
# ITS edge baseline
res = train_svm_from_precomputed_dir(
    precomp_dir=ITS_DIRS["edge"],
    feature_key="its_wl",
    subset_ids=subset_ids,
    n=n, test_size=test_size, C=C, seed=seed,
    verbose=True,
)
add_result(
    tag="S4_baseline",
    kernel="ITS–WL",
    mode="edge",
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
)

df_results = pd.DataFrame(results)
df_results[df_results["tag"]=="S4_baseline"][["kernel","mode","n","test_size","accuracy","subset_ids"]]

### 2) Feature Mode Comparison

This section evaluates the influence of different feature extraction modes on classification accuracy.
Edge-, vertex-, and shortest-path-based WL features are compared while keeping all other parameters fixed.
The experiment highlights which structural information is most informative for reaction classification.

In [ ]:
tag = "S5_modes"
n = 600
test_size = 0.2

for mode in ["vertex","edge","sp"]:
    # DRF
    res = train_svm_from_precomputed_dir(
        precomp_dir=DRF_DIRS[mode], feature_key="drf_wl",
        subset_ids=subset_ids, n=n, test_size=test_size, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S5_modes",
    kernel="DRF–WL",
    mode=mode,
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
)
    # ITS
    res = train_svm_from_precomputed_dir(
        precomp_dir=ITS_DIRS[mode], feature_key="its_wl",
        subset_ids=subset_ids, n=n, test_size=test_size, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S5_modes",
    kernel="ITS–WL",
    mode=mode,
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
)

df_results = pd.DataFrame(results)
df_results[df_results["tag"]==tag][["kernel","mode","n","test_size","accuracy","subset_ids"]].sort_values(["kernel","mode"])

### 3) Effect of Dataset Size

To study the scalability and robustness of the kernel-based approach, the dataset size is varied while
keeping the kernel configuration constant. This experiment shows how classification performance changes
as more training data becomes available.

In [ ]:
tag = "S6_size"
test_size = 0.2

for n in [200, 600, 1200]:
    res = train_svm_from_precomputed_dir(
        precomp_dir=DRF_DIRS["edge"], feature_key="drf_wl",
        subset_ids=subset_ids, n=n, test_size=test_size, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S6_size",
    kernel="DRF–WL",
    mode="edge",
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
    )

    res = train_svm_from_precomputed_dir(
        precomp_dir=ITS_DIRS["edge"], feature_key="its_wl",
        subset_ids=subset_ids, n=n, test_size=test_size, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S6_size",
    kernel="ITS–WL",
    mode="edge",
    n=n,
    test_size=test_size,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
    )

df_results = pd.DataFrame(results)
df_results[df_results["tag"]==tag][["kernel","n","accuracy","subset_ids"]].sort_values(["kernel","n"])

### 4) Effect of Train/Test Split

This experiment investigates the sensitivity of the SVM classifier to different train/test splits.
By increasing the proportion of test data, we assess the stability and generalization capability of the
kernel-based model.

In [ ]:
tag = "S7_splits"
n = 600

for ts in [0.1, 0.2, 0.3, 0.4]:
    res = train_svm_from_precomputed_dir(
        precomp_dir=DRF_DIRS["edge"], feature_key="drf_wl",
        subset_ids=subset_ids, n=n, test_size=ts, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S7_splits",
    kernel="DRF–WL",
    mode="edge",
    n=n,
    test_size=ts,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
    )
    add_result(tag, "DRF–WL", "edge", n, ts, C, seed, res, subset_ids=subset_ids)

    res = train_svm_from_precomputed_dir(
        precomp_dir=ITS_DIRS["edge"], feature_key="its_wl",
        subset_ids=subset_ids, n=n, test_size=ts, C=C, seed=seed,
        verbose=False,
    )
    add_result(
    tag="S7_splits",
    kernel="ITS–WL",
    mode="edge",
    n=n,
    test_size=ts,
    C=C,
    seed=seed,
    res=res,
    subset_ids=subset_ids,
    )

df_results = pd.DataFrame(results)
df_results[df_results["tag"]==tag][["kernel","test_size","accuracy","subset_ids"]].sort_values(["kernel","test_size"])

## Summary of Classification Results

This section summarizes the classification results obtained across all experiments.
The comparison highlights the strengths and limitations of different kernel representations, feature modes,
and dataset configurations, and provides an overall assessment of the kernel-based reaction classification approach.

In [ ]:
df_results = pd.DataFrame(results)

# Überblick
df_results[["tag","kernel","mode","n","test_size","accuracy","subset_ids"]].sort_values(["tag","kernel","mode"]).head(30)

# Optional Plot (wenn deine Funktion existiert)
plot_experiment_results(df_results)

In [ ]:
figs = plot_experiment_results(df_results)
for _, fig in figs.items():
    fig.show()